# Mapping GPX File Data with Python and Mapbox

## Import modules

In [1]:
import plotly.express as px
import numpy as np
import pandas as pd
from gpx_converter import Converter

## Convert local GPX file to Dataframe

In [2]:
wonderland = Converter(input_file='olallie_to_indian_bar.gpx').gpx_to_dataframe()
wonderland

,time,latitude,longitude,altitude
0,2020-09-13 16:15:23+00:00,46.775701,-121.588259,1192.218506
1,2020-09-13 16:15:57+00:00,46.775641,-121.588441,1193.076660
2,2020-09-13 16:16:54+00:00,46.775625,-121.588638,1194.238281
3,2020-09-13 16:17:13+00:00,46.775657,-121.588840,1195.333740
4,2020-09-13 16:17:29+00:00,46.775551,-121.589003,1196.425537
...,...,...,...,...
595,2020-09-13 20:30:50+00:00,46.826005,-121.639022,1567.863037
596,2020-09-13 20:31:22+00:00,46.826119,-121.639131,1565.868896
597,2020-09-13 20:32:35+00:00,46.826213,-121.639276,1563.004883
598,2020-09-13 20:32:55+00:00,46.826098,-121.639396,1565.362061


## Format Data

In [3]:
# Convert GMT to PST and format
wonderland.time = wonderland.time.apply(lambda x: x.tz_convert('US/Pacific'))
wonderland.time = wonderland.time.dt.strftime('%I:%M %p')
wonderland.head(5)


,time,latitude,longitude,altitude
0,09:15 AM,46.775701,-121.588259,1192.218506
1,09:15 AM,46.775641,-121.588441,1193.076660
2,09:16 AM,46.775625,-121.588638,1194.238281
3,09:17 AM,46.775657,-121.588840,1195.333740
4,09:17 AM,46.775551,-121.589003,1196.425537


In [4]:
# Convert altitude from meters to feet
wonderland["alt_ft"] = round(wonderland.altitude * 3.280839895)
wonderland["alt_ft"] = wonderland.alt_ft.astype('int')
wonderland.head(5)

,time,latitude,longitude,altitude,alt_ft
0,09:15 AM,46.775701,-121.588259,1192.218506,3911
1,09:15 AM,46.775641,-121.588441,1193.076660,3914
2,09:16 AM,46.775625,-121.588638,1194.238281,3918
3,09:17 AM,46.775657,-121.588840,1195.333740,3922
4,09:17 AM,46.775551,-121.589003,1196.425537,3925


## Build Map

In [5]:
fig = px.scatter_mapbox(
    wonderland, 
    lat="latitude", 
    lon="longitude",
    hover_name="time",
    hover_data={
        "latitude": ":.2f",
        "longitude": ":.2f",
        "alt_ft": ":, ft"
    },
    zoom=11, 
    height=500,
)

fig.update_layout(
    margin=dict(r=0, t=0, l=0, b=0),
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])

fig.update_traces(hovertemplate='<b>%{hovertext}</b><br><br>(%{customdata[0]:.2f}, %{customdata[1]:.2f})<br>Elev. %{customdata[2]:} .ft<extra></extra>')

fig.show()